In [1]:
import spacy
import numpy as np
import pandas as pd
from stopwords import ENGLISH_STOP_WORDS
import warnings
# from __future__ import unicode_literals
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
#en_nlp = spacy.en.English()
en_nlp=spacy.load('en')

In [3]:
def spacy_get_vec(sentence):
    vec = np.zeros(96)
    doc = en_nlp((sentence))
    for word in doc:
        #if word.lower_ in ENGLISH_STOP_WORDS:
        #    continue
        vec += word.vector
    return vec

In [4]:
lines = open('./question.txt').readlines()
vecs = []
intents = []
idfs = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)
    #idfs.append(get_idf(sentence))

df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(96)])
#df['idf'] = idfs
df['intents'] = intents
df.intents = df.intents.astype('category')

In [5]:
df.head(10)

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_87,vec_88,vec_89,vec_90,vec_91,vec_92,vec_93,vec_94,vec_95,intents
0,-1.759996,-8.420905,2.276270,-1.162590,3.844770,2.840463,-0.753364,2.957269,7.470058,10.314044,...,-5.669944,-1.547058,6.627383,-3.218885,3.190382,1.464900,-4.386830,-3.636473,1.751198,name
1,9.556392,-4.143117,10.756072,-1.359590,-12.705752,-3.072276,-5.545203,-1.486757,-0.434437,13.304896,...,-3.842945,-2.357140,18.024444,-2.828745,-4.346139,18.625840,-6.762220,4.041664,1.481203,name
2,6.220093,-5.076341,2.791780,-5.896989,-14.144672,-1.569555,-10.525382,-7.483668,2.840772,10.478342,...,1.345230,-4.289471,14.988673,-3.042850,0.525640,24.159926,-2.263616,2.899887,4.113554,name
3,3.689054,0.810873,4.189868,-6.209874,-10.413410,4.986526,-2.821653,-2.500333,0.190013,11.839452,...,-8.401937,-1.275353,6.801534,-7.838282,3.462574,21.809196,-2.253262,-1.382785,8.578007,name
4,3.607861,4.336752,6.148700,-5.055354,-12.857260,3.716480,3.296282,-3.805648,-4.982093,15.203674,...,-15.804678,3.829497,8.984690,-2.952568,-0.446337,15.480394,0.490406,0.078620,2.116596,name
5,5.417962,-0.033732,5.287983,-6.812062,-13.363475,1.552076,-5.805094,-1.555635,-1.880611,13.349326,...,-4.537413,-1.163738,8.648477,-5.198575,0.031843,15.966703,0.642479,8.165331,0.568275,name
6,5.659891,-6.080133,7.118525,-4.139761,-9.131500,-1.621250,-6.766408,-7.214301,6.137925,15.765751,...,-0.745241,-3.616721,14.099648,-2.818982,5.383953,15.527542,-3.013386,-1.775828,1.177742,name
7,10.605276,1.534121,7.547316,-3.753668,-6.778317,-1.711190,-2.080208,-0.661567,-5.063860,11.907645,...,0.057007,5.451145,14.363190,-1.573192,-4.511716,17.581675,-4.329326,0.226867,2.271344,name
8,6.220093,-5.076341,2.791780,-5.896989,-14.144672,-1.569555,-10.525382,-7.483668,2.840772,10.478342,...,1.345230,-4.289471,14.988673,-3.042850,0.525640,24.159926,-2.263616,2.899887,4.113554,name
9,5.765600,-5.092359,10.626873,-9.261091,-1.491605,7.644727,-2.877099,-2.109652,3.450187,11.730363,...,-7.019221,1.508247,-0.149125,-0.155668,4.082173,16.442611,-2.727755,1.052329,6.784730,name


In [6]:
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1:].values.ravel()

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.20)

In [10]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X_train, y_train)
print(logit_model.score(X_val, y_val))


0.75


/home/vidooly/shubham/nlp_env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/vidooly/shubham/nlp_env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gradboost = GradientBoostingClassifier(n_estimators=600, max_depth=30, learning_rate=0.01)
gradboost.fit(X_train, y_train)
print(gradboost.score(X_val, y_val))


0.8333333333333334


In [11]:
sent = 'do you live in the computer'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[ -8.81953237e+00  -8.81157917e+00  -7.96046149e-04  -8.68235072e+00
   -8.70572591e+00  -8.71730751e+00]]
[u'location']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [12]:
sent = 'how does your mom call you'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[ -9.17711136e+00  -9.16915816e+00  -9.02683503e+00  -9.04073123e+00
   -5.60872124e-04  -9.07488650e+00]]
[u'name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [13]:
sent = 'how does my mom call me'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[-9.39218666 -9.38423345 -9.24191033 -9.21899022 -1.33852408 -0.30461918]]
[u'users_name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [12]:
gradboost.classes_

array(['bot', 'greet_response', 'location', 'marriage', 'name',
       'users_name'], dtype=object)

In [23]:
from sklearn.svm import LinearSVC
svcmodel=LinearSVC(multi_class='ovr',C=0.1)
svcmodel.fit(X_train,y_train)

/home/vidooly/shubham/nlp_env/lib/python3.5/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [24]:
svcmodel.score(X_val,y_val)

0.75

In [17]:
sent = 'do you live in the computer'
#print(svcmodel.predict_log_proba(spacy_get_vec(sent)))
print(svcmodel.predict(spacy_get_vec(sent)))

[u'location']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [18]:
sent = 'how does your mom call you'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[ -9.17711136e+00  -9.16915816e+00  -9.02683503e+00  -9.04073123e+00
   -5.60872124e-04  -9.07488650e+00]]
[u'name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [15]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB(alpha=1.7)
mnb.fit(abs(X_train), y_train)

MultinomialNB(alpha=1.7, class_prior=None, fit_prior=True)

In [16]:
mnb.score(abs(X_val), y_val)

0.6666666666666666

In [25]:
gradboost.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=30,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=600,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [26]:
from sklearn.externals import joblib
joblib.dump(gradboost, 'question.pkl')

['question.pkl']